In [ ]:
### Develop a program to automatically count the number of malarial
### parasites within a blood smear of red blood cells using Scikit-image

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu
from skimage.morphology import binary_erosion, square, remove_small_holes
from skimage.measure import label, regionprops
from skimage.filters import rank
from skimage.morphology import disk
import matplotlib.pyplot as plt
import numpy as np

# Clone the Lecture03 repository to access the imaging data file
#!git clone https://github.com/BMEn-datascience/Lecture03.git

# Read in the rbc_malaria.png file (color)
image = imread('Lecture03/rbc_malaria.png')
plt.figure(figsize = (8,4))
plt.subplot(1,2,1)
plt.imshow(image)
plt.title('Color Image')

# Convert to grayscale (helpful for image processing)
image_gray = rgb2gray(image[:,:,:3])
plt.subplot(1,2,2)
plt.imshow(image_gray, cmap='gray')
plt.title('Grayscale Image')
plt.tight_layout()
plt.show()

# Or, read in the rbc_malara.png file as grayscale directly
# image_gray = imread('Lecture03/rbc_malaria.png', as_gray=True)
# plt.imshow(image, cmap='gray')
# plt.show()

# Create a 18x18 averaging filter
selem = disk(18)  # Create a circular structuring element with radius 18

# Apply the averaging filter to the grayscale image
image_averaged = rank.mean(image_gray, selem)

# Display the average filtered image
plt.figure(figsize = (8,4))
plt.subplot(1,2,1)
plt.imshow(image_gray, cmap='gray')
plt.title('Original Image')
plt.subplot(1,2,2)
plt.imshow(image_averaged, cmap='gray')
plt.title('Averaged Filtered Image')
plt.tight_layout()
plt.show()

# Flatten the image into a 1D array to inspect the pixel intensity distribution
pixel_intensities = image_averaged.flatten()

# Create a histogram of pixel intensities
plt.hist(pixel_intensities, bins=256, range=(0, 256))
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.title('Histogram of Pixel Intensities')
plt.show()



### Segment and count (automatically!) the number of parasites in the image

# Create a binary image based on manual thresholding
image_binary = image_averaged < 150  # Adjust the threshold as needed
plt.imshow(image_binary, cmap='gray')
plt.show()

# Create a binary image based on automated thresholding
thresh = threshold_otsu(image_gray)
image_binary_auto = image_gray < thresh
plt.imshow(image_binary_auto, cmap='gray')
plt.show()

# Label connected regions (parasites)
label_image = label(image_binary)
regions = regionprops(label_image)

# Count the number of parasites
num_parasites = len(regions)

print("Number of parasites:", num_parasites)



### Perform morphological processing on the red blood cell segmentations

# Remove small holes from the red blood cells
bw = remove_small_holes(image_binary_auto, area_threshold=1000)

# Erode the binary image so cells are not connected
bwe = binary_erosion(bw, square(36))

# Show the original binary image and processed image
plt.figure(figsize = (8,4))
plt.subplot(1,2,1)
plt.imshow(image_binary_auto, cmap='gray')
plt.title('Binary Image')
plt.subplot(1,2,2)
plt.imshow(bwe, cmap='gray')
plt.title('Separated Cells')
plt.tight_layout()
plt.show()
